In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# model file path
model_path = '/content/drive/My Drive/GA/Capstone/capstone/models/celtics_uncropped_model.h5'

# print statements to evaluate whether the model loaded correctly
if os.path.exists(model_path):
    print("Model file found!")
else:
    print("Model file not found. Please check the path or upload the file.")


Model file found!


In [3]:
!pip install streamlit
!pip install pyngrok
!pip install ultralytics
!pip install easyocr
!pip install opencv-python-headless



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metada

In [4]:

from pyngrok import ngrok

# ngrok authtoken
!ngrok authtoken CONNOR_AUTHTOKEN # no u can't have it

# creating a public URL using ngrok
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ERROR:pyngrok.process.ngrok:t=2024-08-25T21:18:50+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: CONNOR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-08-25T21:18:50+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: CONNOR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: CONNOR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
with open('app.py', 'w') as f:
    f.write(f'''
import streamlit as st
import cv2
from PIL import Image
import numpy as np
import easyocr
import tensorflow as tf
from ultralytics import YOLO
import pandas as pd

def main():
    # loading models and data
    yolo = YOLO('yolov8n.pt')  # Base YOLOv8 model
    celtic_classifier = tf.keras.models.load_model('{model_path}')
    ocr_reader = easyocr.Reader(['en'])
    roster_data = pd.read_csv('/content/drive/MyDrive/GA/Capstone/capstone/data/BOS_RosterData.csv')

    # preprocessing the image by converting it to grayscale, applying a gaussian blur, and using adaptive thresholding
    def preprocess_image(image):
        # Converting to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # applying a gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # applying adaptive thresholding
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)

        # using a morphological operation to further clean up the image
        kernel = np.ones((3, 3), np.uint8)
        processed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        return processed

    # cross-referencing the detected jersey number with the roster data
    def cross_reference_with_roster(jersey_number, year):
        player_info = roster_data[(roster_data['Year'] == int(year)) &
                                  (roster_data['JerseyNumber'] == jersey_number)]
        if not player_info.empty:
            player_name = player_info['PlayerName'].values[0]
            player_url = player_info['PlayerURL'].values[0]
            return player_name, player_url
        else:
            return None, f"No match between jersey number and roster data for season {{year}} was found."

    # processing the uploaded image, running the YOLO model, and applying OCR
    def process_image(image, year):
        image_np = np.array(image)
        results = yolo(image_np)
        person_detections = [box for box in results[0].boxes.data if int(box[5]) == 0]  # Class 0 for persons

        for box in person_detections:
            xmin, ymin, xmax, ymax = map(int, box[:4])
            cropped_person = image_np[ymin:ymax, xmin:xmax]

            cropped_person_resized = cv2.resize(cropped_person, (224, 224))
            cropped_person_resized = np.expand_dims(cropped_person_resized, axis=0) / 255.0

            is_celtic = celtic_classifier.predict(cropped_person_resized)[0][0]
            if is_celtic > 0.5:
                # draws the bounding box on the original image
                cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)  # Red box

                # performing initial OCR
                digits = ocr_reader.readtext(cropped_person)
                jersey_number = ''.join([digit[1] for digit in digits if digit[1].isdigit()])

                # if no jersey number is detected, applying preprocessing and trying OCR again
                if not jersey_number:
                    preprocessed_image = preprocess_image(cropped_person)
                    digits = ocr_reader.readtext(preprocessed_image)
                    jersey_number = ''.join([digit[1] for digit in digits if digit[1].isdigit()])

                player_name, message = cross_reference_with_roster(jersey_number, year)
                if player_name:
                    return Image.fromarray(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)), player_name, message
                else:
                    return Image.fromarray(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)), "Celtic player identified", message

        # returning the original image if no Celtic player is detected
        return image, "No Celtics player found", ""

    # Streamlit App
    st.title("Celtic Player Identifier")

    # including the 'webp' in the allowed file types
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg", "webp"])
    year = st.text_input("Enter the season year (optional)")

    if uploaded_file is not None:
        image = Image.open(uploaded_file)

        # Displaying the original image
        st.image(image, caption='Uploaded Image.', use_column_width=True)

        # Resizing the image to fit within a 1080p screen height (max height of 300 pixels)
        max_height = 300
        width, height = image.size
        aspect_ratio = width / height
        new_width = int(max_height * aspect_ratio)

        # Resizing the image
        resized_image = image.resize((new_width, max_height))

        if st.button('Identify Player'):
            if year.isdigit():
                processed_image, player_name, message = process_image(resized_image, year)
                st.image(processed_image, caption='Processed Image with Bounding Box', use_column_width=True)
                st.write(player_name)
                st.write(message)
            else:
                st.error("Please enter a valid year.")

if __name__ == '__main__':
    main()
''')

In [ ]:
# run the Streamlit app
!streamlit run app.py &>/dev/null&
